In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,f1_score,precision_score
from sklearn.metrics import cohen_kappa_score,hamming_loss,precision_recall_curve
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from DBN.models import SupervisedDBNClassification
import  tensorflow
import  pandas as pd
#import eli5
#from eli5.sklearn import PermutationImportance
#import shap
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
print(tensorflow.__version__)

In [ ]:
# Loading dataset
# digits = load_digits()
# X, Y = digits.data, digits.target
data = pd.read_csv(r'train4.csv',sep=',')
#data_= pd.read_csv(r'test.csv',sep=',')
label_index = 7
dim  = 7
# data[label_index] = data[label_index].map({1:1, 2:-1})
data = data.sample(frac=1.0)
data = data.reset_index()
x = data.drop(['landslide'],axis=1)
print(x.shape)
x = x.drop(['index'],axis=1)
x = np.array(x)
print(x.shape)
std=MinMaxScaler((0,1))
x=std.fit_transform(x)
y = data['landslide']
y = pd.to_numeric( y, errors='coerce').fillna('0').astype('int32')
# Data scaling
# X = (X / 16).astype(np.float32)

In [ ]:
#test
data_= pd.read_csv(r'test4.csv',sep=',')
data_.dropna(inplace=True)
data_ = data_.sample(frac=1.0)
data_ = data_.reset_index()
x_ = data_.drop(['landslide'],axis=1)
x_ = x_.drop(['index'],axis=1)
x_ = x_.drop(['pre1'],axis=1)
x_ = np.array(x_)
res=x_
print(x_.shape)
std=MinMaxScaler((0,1))
x_=std.fit_transform(x_)
y_ = data_['landslide']
y_ = pd.to_numeric( y_, errors='coerce').fillna('0').astype('int32')

In [ ]:
# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)#random_state=0
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[100, 100],
                                         learning_rate_rbm=0.07, 
                                         learning_rate=0.15,
                                         n_epochs_rbm=16,
                                         n_iter_backprop=800,
                                         batch_size=20,
                                         activation_function='relu',
                                         dropout_p=0)
classifier.fit(X_train[:,2:], Y_train)

In [ ]:
# Test
Y_pred = classifier.predict(X_test[:,2:])

#Y_pred_p = classifier.predict_proba(X_test[:,2:])[:,1]
Y_pred_p = classifier.predict_proba_dict(X_test[:,2:])[:]

Y_pred = np.array(Y_pred)
Y_test_1 = np.array(Y_test)
Y_pred_p = np.array(Y_pred_p)
Y_pred_1 = np.reshape(Y_pred,(len(Y_pred),1))
Y_test_1 = np.reshape(Y_test_1,(len(Y_test_1),1))
Y_pred_p = np.reshape(Y_pred_p,(len(Y_pred_p),1))
#Y_pred_1
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))
#print('Done.\nprecision_score: %f' % precision_score(Y_test, Y_pred))
#print('Done.\nrecall_score: %f' % recall_score(Y_test, Y_pred))
#print('Done.\nf1_score: %f' % f1_score(Y_test, Y_pred))
print('Done.\nconfusion_matrix: ' )
print(confusion_matrix(Y_test, Y_pred))
print('Done.\ncohen_kappa_score: %f' % cohen_kappa_score(Y_test, Y_pred))
print('Done.\nhamming_loss: %f' % hamming_loss(Y_test, Y_pred))
#fpr,tpr,threshold= roc_curve(Y_test,Y_pred_p)
#print('Done.\nauc: %f' % auc(fpr, tpr))
target_names = ['class 0','class 1','class 2', 'class 3']
print(classification_report(Y_test, Y_pred, target_names=target_names))

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn import metrics
from scipy import interp
from itertools import cycle
n_class = [0,1,2,3]
y_one_hot = label_binarize(Y_test, n_class)
y_two_hot = label_binarize(Y_pred_1, n_class)
n_classes = y_one_hot.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_one_hot[:, i].ravel(), y_two_hot[:, i].ravel())
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
#method_one
fpr["micro"], tpr["micro"], _ = roc_curve(y_one_hot.ravel(), y_two_hot.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])
print("micro：",roc_auc["micro"])
#method_two
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])
print("macro：",roc_auc["macro"])

#plot
lw=2
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.3f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
Y_pred = classifier.predict(X_train[:,2:])
#Y_pred_p = classifier.predict_proba_dict(X_test[:,2:])[:]

Y_pred = np.array(Y_pred)
Y_test_1 = np.array(Y_train)
#Y_pred_p = np.array(Y_pred_p)
Y_pred_1 = np.reshape(Y_pred,(len(Y_pred),1))
Y_test_1 = np.reshape(Y_test_1,(len(Y_test_1),1))
#Y_pred_p = np.reshape(Y_pred_p,(len(Y_pred_p),1))
print('Done.\nconfusion_matrix: ' )
print(confusion_matrix(Y_train, Y_pred))
print('Done.\ncohen_kappa_score: %f' % cohen_kappa_score(Y_train, Y_pred))

n_class = [0,1,2,3]
y_one_hot = label_binarize(Y_test_1, n_class)
y_two_hot = label_binarize(Y_pred_1, n_class)
n_classes = y_one_hot.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_one_hot[:, i].ravel(), y_two_hot[:, i].ravel())
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
#method_one
fpr["micro"], tpr["micro"], _ = roc_curve(y_one_hot.ravel(), y_two_hot.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])
print("micro：",roc_auc["micro"])
#method_two
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])
print("macro：",roc_auc["macro"])

#plot
lw=2
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.3f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Y_test_1
all_save_data = np.concatenate((X_test,Y_test_1),axis=1)
mode=0
if mode==0:
    all_save_data = np.concatenate((all_save_data,Y_pred_1),axis=1)
else:
    all_save_data = np.concatenate((all_save_data,Y_pred_p),axis=1)
all_save_data_pd = pd.DataFrame(all_save_data,columns=['x','y','road','PGA',
 'river','fault','lithology','NDVI','Ht','landslide','predict'])
 # np.savetxt('data.csv',all_save_data,delimiter=',')
all_save_data_pd.to_csv('churoc.csv',index=True)
# y_test_1.shape

In [ ]:
# Test
y_pred = classifier.predict(x_[:,2:])
#y_pred_p = classifier.predict_proba(x_[:,2:])[:,1]
y_pred = np.array(y_pred)
y_test_1 = np.array(y_)
#y_pred_p = np.array(y_pred_p)
y_pred_1 = np.reshape(y_pred,(len(y_pred),1))
y_test_1 = np.reshape(y_test_1,(len(y_test_1),1))
#y_pred_p = np.reshape(y_pred_p,(len(y_pred_p),1))

In [ ]:
all_save_data = np.concatenate((res,y_test_1),axis=1)
mode=0
if mode==0:
    all_save_data = np.concatenate((all_save_data,y_pred_1),axis=1)
else:
    all_save_data = np.concatenate((all_save_data,y_pred_p),axis=1)
all_save_data_pd = pd.DataFrame(all_save_data,columns=['x','y','road','PGA',
 'river','fault','lithology','NDVI','Ht','landslide','predict'])
 # np.savetxt('data.csv',all_save_data,delimiter=',')
all_save_data_pd.to_csv('result4.csv',index=True)
# y_test_1.shape